In [1]:
# filename: output.ipynb
# purpose: generate output data

# OHT noise, normal and outlier data generation

### Outlier detection method
- Outlier data will be detected and generated based on Moving Average and Moving Standard Deviation 

### Processing flow
- Read Raw table, ohtraw from duckdb filedb, which was created in previous step, parse notebook
- Create a Work table, work in memory duckdb with additional columns MVAVG_,MVSTD_,MVSIG_,FLAG columns.
- Calculate Moving Average, Standard Devidation
- Calculate Sigma value of the column data based on Moving Average and Standard Deviation
- Calcuate FLAG column value based on Sigma value
- Fetch the work table into work dataframe
- Split noise and normal dataframe from work dataframe based on FLAG value
- Create outlier dataframe based on normal dataframe
- Update outlier dataframe by applying outlier pattern 
- Save noise, normal, outler dataframe to duckdb tables for later graphing.
- Save noise, normal, outler csvfle
- Check csvfile size 

In [2]:
# packages
import time
import pathlib
import textwrap

import humanfriendly as human
import duckdb

import ohtconf as conf
import ohtcomm as comm

## Main

In [3]:
mainstart = time.time()

### Prepare in-memory work table base on in-file raw table 

In [4]:
# open in-memory db

con = duckdb.connect(database=":memory:")

In [5]:
# create in-memory work table

query = "DROP TABLE IF EXISTS work"
con.execute(query)

coldef = ""
for name, dtype in zip(conf.COLUMN_NAMES, conf.COLUMN_DBTYPES):
    if coldef:
        coldef += ", " + name + " " + dtype
    else:
        coldef += name + " " + dtype

for col in conf.COLUMN_GRAPH:
    coldef += ", " + conf.MVAVG + col + " " + "FLOAT"
    coldef += ", " + conf.MVSTD + col + " " + "FLOAT"
    coldef += ", " + conf.MVSIG + col + " " + "INTEGER"

coldef += ", " + f"{conf.COLUMN_FLAG}  INTEGER"

query = f"CREATE TABLE work ( {coldef} )"
print(textwrap.fill(query, width=120))

con.execute(query)

CREATE TABLE work ( DATETM TIMESTAMP, TEMPER FLOAT, PM1 INTEGER, PM2_5 INTEGER, PM10 INTEGER, CO INTEGER, NH3 INTEGER,
CT1 FLOAT, CT2 FLOAT, CT3 FLOAT, CT4 FLOAT, MVAVG_TEMPER FLOAT, MVSTD_TEMPER FLOAT, MVSIG_TEMPER INTEGER, MVAVG_PM1
FLOAT, MVSTD_PM1 FLOAT, MVSIG_PM1 INTEGER, MVAVG_PM2_5 FLOAT, MVSTD_PM2_5 FLOAT, MVSIG_PM2_5 INTEGER, MVAVG_PM10 FLOAT,
MVSTD_PM10 FLOAT, MVSIG_PM10 INTEGER, MVAVG_CO FLOAT, MVSTD_CO FLOAT, MVSIG_CO INTEGER, MVAVG_NH3 FLOAT, MVSTD_NH3
FLOAT, MVSIG_NH3 INTEGER, MVAVG_CT1 FLOAT, MVSTD_CT1 FLOAT, MVSIG_CT1 INTEGER, MVAVG_CT2 FLOAT, MVSTD_CT2 FLOAT,
MVSIG_CT2 INTEGER, MVAVG_CT3 FLOAT, MVSTD_CT3 FLOAT, MVSIG_CT3 INTEGER, MVAVG_CT4 FLOAT, MVSTD_CT4 FLOAT, MVSIG_CT4
INTEGER, FLAG  INTEGER )


In [6]:
# attach in-file db, raw table was prepared in the previous step, parse

con.execute(f"ATTACH DATABASE '{conf.DBFILE}' AS filedb (READ_ONLY)")

In [7]:
# insert into in-memory work table from in-file raw table with additional Moving Avg,Std and Flag=0

coldef = ", ".join(conf.COLUMN_NAMES)

for col in conf.COLUMN_GRAPH:
    coldef += (
        ", "
        + f"AVG({col}) OVER (ORDER BY {conf.COLUMN_NAMES[0]} ROWS BETWEEN {conf.POINTS['MOVING']} PRECEDING AND CURRENT ROW) AS {conf.MVAVG}{col}"
    )
    coldef += (
        ", "
        + f"STDDEV({col}) OVER (ORDER BY {conf.COLUMN_NAMES[0]} ROWS BETWEEN {conf.POINTS['MOVING']} PRECEDING AND CURRENT ROW) AS {conf.MVSTD}{col}"
    )
    coldef += ", 0"  # mvsig_
coldef += ", 0"  # flag

query = f"INSERT INTO work SELECT {coldef} FROM filedb.{conf.TABNAME_RAW} ORDER BY {conf.COLUMN_NAMES[0]}"
print(textwrap.fill(query, width=120))

con.execute(query)

INSERT INTO work SELECT DATETM, TEMPER, PM1, PM2_5, PM10, CO, NH3, CT1, CT2, CT3, CT4, AVG(TEMPER) OVER (ORDER BY DATETM
ROWS BETWEEN 600 PRECEDING AND CURRENT ROW) AS MVAVG_TEMPER, STDDEV(TEMPER) OVER (ORDER BY DATETM ROWS BETWEEN 600
PRECEDING AND CURRENT ROW) AS MVSTD_TEMPER, 0, AVG(PM1) OVER (ORDER BY DATETM ROWS BETWEEN 600 PRECEDING AND CURRENT
ROW) AS MVAVG_PM1, STDDEV(PM1) OVER (ORDER BY DATETM ROWS BETWEEN 600 PRECEDING AND CURRENT ROW) AS MVSTD_PM1, 0,
AVG(PM2_5) OVER (ORDER BY DATETM ROWS BETWEEN 600 PRECEDING AND CURRENT ROW) AS MVAVG_PM2_5, STDDEV(PM2_5) OVER (ORDER
BY DATETM ROWS BETWEEN 600 PRECEDING AND CURRENT ROW) AS MVSTD_PM2_5, 0, AVG(PM10) OVER (ORDER BY DATETM ROWS BETWEEN
600 PRECEDING AND CURRENT ROW) AS MVAVG_PM10, STDDEV(PM10) OVER (ORDER BY DATETM ROWS BETWEEN 600 PRECEDING AND CURRENT
ROW) AS MVSTD_PM10, 0, AVG(CO) OVER (ORDER BY DATETM ROWS BETWEEN 600 PRECEDING AND CURRENT ROW) AS MVAVG_CO, STDDEV(CO)
OVER (ORDER BY DATETM ROWS BETWEEN 600 PRECEDING AND CU

In [8]:
# detach filedb

con.execute("DETACH DATABASE filedb")

In [9]:
# Update NULL value after window function
query = f"SELECT * FROM work ORDER BY {conf.COLUMN_NAMES[0]}"
dfwork = con.execute(query).df()
dfwork.bfill(inplace=True)

# recreate work table base one work dataframe
con.execute("DROP TABLE IF EXISTS work")
con.execute("CREATE TABLE work AS SELECT * FROM dfwork")

In [10]:
# calculate Sigma value

coldef = ""
for col in conf.COLUMN_GRAPH:
    setdef = textwrap.dedent(f"""{conf.MVSIG}{col} = CASE 
                  WHEN {col} >= ({col} - 1 * {conf.MVSTD}{col}) AND {col} <= ({col} + 1 * {conf.MVSTD}{col})  then 1
                  WHEN {col} >= ({col} - 2 * {conf.MVSTD}{col}) AND {col} <= ({col} + 2 * {conf.MVSTD}{col})  then 2
                  WHEN {col} >= ({col} - 3 * {conf.MVSTD}{col}) AND {col} <= ({col} + 3 * {conf.MVSTD}{col})  then 3
                  WHEN {col} >= ({col} - 4 * {conf.MVSTD}{col}) AND {col} <= ({col} + 4 * {conf.MVSTD}{col})  then 4
                  WHEN {col} >= ({col} - 5 * {conf.MVSTD}{col}) AND {col} <= ({col} + 5 * {conf.MVSTD}{col})  then 5
                  ELSE 6
                  END""")
    if not coldef:
        coldef = setdef
    else:
        coldef += f", {setdef}"

query = f"UPDATE work SET {coldef}"
print(textwrap.fill(query, width=120))

con.execute(query)

UPDATE work SET MVSIG_TEMPER = CASE                    WHEN TEMPER >= (TEMPER - 1 * MVSTD_TEMPER) AND TEMPER <= (TEMPER
+ 1 * MVSTD_TEMPER)  then 1                   WHEN TEMPER >= (TEMPER - 2 * MVSTD_TEMPER) AND TEMPER <= (TEMPER + 2 *
MVSTD_TEMPER)  then 2                   WHEN TEMPER >= (TEMPER - 3 * MVSTD_TEMPER) AND TEMPER <= (TEMPER + 3 *
MVSTD_TEMPER)  then 3                   WHEN TEMPER >= (TEMPER - 4 * MVSTD_TEMPER) AND TEMPER <= (TEMPER + 4 *
MVSTD_TEMPER)  then 4                   WHEN TEMPER >= (TEMPER - 5 * MVSTD_TEMPER) AND TEMPER <= (TEMPER + 5 *
MVSTD_TEMPER)  then 5                   ELSE 6                   END, MVSIG_PM1 = CASE                    WHEN PM1 >=
(PM1 - 1 * MVSTD_PM1) AND PM1 <= (PM1 + 1 * MVSTD_PM1)  then 1                   WHEN PM1 >= (PM1 - 2 * MVSTD_PM1) AND
PM1 <= (PM1 + 2 * MVSTD_PM1)  then 2                   WHEN PM1 >= (PM1 - 3 * MVSTD_PM1) AND PM1 <= (PM1 + 3 *
MVSTD_PM1)  then 3                   WHEN PM1 >= (PM1 - 4 * MVSTD_PM1) AND PM1 <= 

In [11]:
# calculate Flag=1 based on configured sigma value

coldef = ""
for col in conf.COLUMN_GRAPH:
    if not coldef:
        coldef = f"{conf.MVSIG}{col} > {conf.SIGMA_NOISE}"
    else:
        coldef += f" OR {conf.MVSIG}{col} > {conf.SIGMA_NOISE}"

query = f"UPDATE work SET {conf.COLUMN_FLAG}=1 WHERE {coldef}"
print(textwrap.fill(query, width=120))

con.execute(query)

UPDATE work SET FLAG=1 WHERE MVSIG_TEMPER > 2 OR MVSIG_PM1 > 2 OR MVSIG_PM2_5 > 2 OR MVSIG_PM10 > 2 OR MVSIG_CO > 2 OR
MVSIG_NH3 > 2 OR MVSIG_CT1 > 2 OR MVSIG_CT2 > 2 OR MVSIG_CT3 > 2 OR MVSIG_CT4 > 2


### Prepare noise, normal, outlier dataframe

In [12]:
# fetch table

dfnoise = con.execute(f"SELECT * FROM work WHERE {conf.COLUMN_FLAG}=1 ORDER BY {conf.COLUMN_NAMES[0]}").df()

dfnorm = con.execute(f"SELECT * FROM work WHERE {conf.COLUMN_FLAG}=0 ORDER BY {conf.COLUMN_NAMES[0]}").df()

dfoutl = dfnorm.iloc[: round(len(dfnorm) * conf.OUTLIER_RATIO)]  # normal:outlier = 3:1 (6:2)
dfoutl = dfoutl.sort_values(by=conf.COLUMN_NAMES[0])
dfoutl = dfoutl.reset_index(drop=True)

print(f"row count, noise={len(dfnoise)}, normal={len(dfnorm)}, outlier={len(dfoutl)}")

row count, noise=0, normal=108000, outlier=36000


In [13]:
# data before outlier updating
dfoutl.head()

,DATETM,TEMPER,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4,MVAVG_TEMPER,MVSTD_TEMPER,MVSIG_TEMPER,MVAVG_PM1,MVSTD_PM1,MVSIG_PM1,MVAVG_PM2_5,MVSTD_PM2_5,MVSIG_PM2_5,MVAVG_PM10,MVSTD_PM10,MVSIG_PM10,MVAVG_CO,MVSTD_CO,MVSIG_CO,MVAVG_NH3,MVSTD_NH3,MVSIG_NH3,MVAVG_CT1,MVSTD_CT1,MVSIG_CT1,MVAVG_CT2,MVSTD_CT2,MVSIG_CT2,MVAVG_CT3,MVSTD_CT3,MVSIG_CT3,MVAVG_CT4,MVSTD_CT4,MVSIG_CT4,FLAG
0,2024-07-29 09:43:18.696,40.799999,10,12,13,161,88,0.8,1.0,0.5,0.6,40.799999,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.800,0.000000,1,1.0,0.0,1,0.50,0.000000,1,0.600000,0.000000,1,0
1,2024-07-29 09:43:18.781,40.799999,10,12,13,161,88,0.8,1.0,0.5,0.6,40.799999,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.800,0.000000,1,1.0,0.0,1,0.50,0.000000,1,0.600000,0.000000,1,0
2,2024-07-29 09:43:18.883,40.799999,10,12,13,161,88,0.8,1.0,0.5,0.5,40.799999,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.800,0.000000,1,1.0,0.0,1,0.50,0.000000,1,0.566667,0.057735,1,0
3,2024-07-29 09:43:18.989,40.799999,10,12,13,161,88,0.9,1.0,0.5,0.6,40.799999,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.825,0.050000,1,1.0,0.0,1,0.50,0.000000,1,0.575000,0.050000,1,0
4,2024-07-29 09:43:19.094,40.799999,10,12,13,161,88,0.8,1.0,0.7,0.6,40.799999,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.820,0.044721,1,1.0,0.0,1,0.54,0.089443,1,0.580000,0.044721,1,0


In [14]:
# Update outlier
_start = time.time()

dfoutl = comm.gen_outlier(dfoutl)

_elapsed = time.time() - _start
print(
    f"get_outlier elapsed time: {human.format_timespan(_elapsed)}"
)  # 8 min 17 sec, 2_031_674 rows, INPUT_MAXSIZE=400MB

outlier count=1 / 36000
get_outlier elapsed time: 8.91 seconds


In [15]:
# round float type value
dfnoise = dfnoise.round(1)
dfnorm = dfnorm.round(1)
dfoutl = dfoutl.round(1)

In [16]:
# set float display format
# pd.set_option("display.float_format", "{:.1f}".format)

In [17]:
# data after outlier updating
dfoutl.head()

,DATETM,TEMPER,PM1,PM2_5,PM10,CO,NH3,CT1,CT2,CT3,CT4,MVAVG_TEMPER,MVSTD_TEMPER,MVSIG_TEMPER,MVAVG_PM1,MVSTD_PM1,MVSIG_PM1,MVAVG_PM2_5,MVSTD_PM2_5,MVSIG_PM2_5,MVAVG_PM10,MVSTD_PM10,MVSIG_PM10,MVAVG_CO,MVSTD_CO,MVSIG_CO,MVAVG_NH3,MVSTD_NH3,MVSIG_NH3,MVAVG_CT1,MVSTD_CT1,MVSIG_CT1,MVAVG_CT2,MVSTD_CT2,MVSIG_CT2,MVAVG_CT3,MVSTD_CT3,MVSIG_CT3,MVAVG_CT4,MVSTD_CT4,MVSIG_CT4,FLAG
0,2024-07-29 09:43:18.696,40.799999,10,12,13,161,88,0.8,1.0,0.5,0.6,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.0,1,0.6,0.0,1,1
1,2024-07-29 09:43:18.781,40.900002,10,12,13,162,88,0.8,1.0,0.5,0.6,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.0,1,0.6,0.0,1,1
2,2024-07-29 09:43:18.883,41.000000,11,13,14,162,89,0.8,1.0,0.6,0.6,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.0,1,0.6,0.1,1,1
3,2024-07-29 09:43:18.989,41.299999,11,13,14,162,89,0.8,1.1,0.6,0.7,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.0,1,0.6,0.1,1,1
4,2024-07-29 09:43:19.094,41.299999,11,14,14,163,89,0.9,1.1,0.6,0.7,40.8,0.0,1,10.0,0.0,1,12.0,0.0,1,13.0,0.0,1,161.0,0.0,1,88.0,0.0,1,0.8,0.0,1,1.0,0.0,1,0.5,0.1,1,0.6,0.0,1,1


### Save dataframe into duckdb table

In [18]:
# save noise
_start = time.time()

comm.save_dftab(dfnoise, conf.TABNAME_NOISE)

_elapsed = time.time() - _start
print(f"save db, noise elapsed time: {human.format_timespan(_elapsed)}")

save db, noise elapsed time: 0.03 seconds


In [19]:
# save normal
_start = time.time()

comm.save_dftab(dfnorm, conf.TABNAME_NORM)

_elapsed = time.time() - _start
print(f"save db, norm elapsed time: {human.format_timespan(_elapsed)}")

save db, norm elapsed time: 0.11 seconds


In [20]:
# save outlier
_start = time.time()

comm.save_dftab(dfoutl, conf.TABNAME_OUTL)

_elapsed = time.time() - _start
print(f"save db, outl elapsed time: {human.format_timespan(_elapsed)}")

save db, outl elapsed time: 0.09 seconds


In [21]:
# close in-memory db
con.close()

### Save dataframe into csvfile

In [22]:
# save noise
_start = time.time()

comm.save_csvfile(dfnoise[conf.COLUMN_NAMES], conf.FILENAME_NOISE, conf.DIROUT)  # only noise

_elapsed = time.time() - _start
print(f"save csvfile, noise elapsed time: {human.format_timespan(_elapsed)}")  # no files for conf.INPUT_MAXSIZE=400 MB

save_csvfile, dataframe has no rows to write, skip
save csvfile, noise elapsed time: 0 seconds


In [23]:
# save normal
_start = time.time()

comm.save_csvfile(dfnorm[conf.COLUMN_NAMES], conf.FILENAME_NORM, conf.DIROUT)

_elapsed = time.time() - _start
print(
    f"save csvfile, norm elapsed time: {human.format_timespan(_elapsed)}"
)  #  8 min. 52 sec, 170 files, 374 MB for conf.INPUT_MAXSIZE=400 MB

saved 1/3 csvfile=sample\dataout\ohtnorm\ohtnorm-001.csv
saved 3/3 files
save csvfile, norm elapsed time: 9.43 seconds


In [24]:
# save outlier
_start = time.time()

comm.save_csvfile(dfoutl[conf.COLUMN_NAMES], conf.FILENAME_OUTL, conf.DIROUT)

_elapsed = time.time() - _start
print(
    f"save csvfile, outl elapsed time: {human.format_timespan(_elapsed)}"
)  # 2 min. 57 sec, 57 files 126 MB for conf.INPUT_MAXSIZE=400 MB

saved 1/1 csvfile=sample\dataout\ohtoutl\ohtoutl-001.csv
saved 1/1 files
save csvfile, outl elapsed time: 3.22 seconds


### Check file size

In [25]:
# check file size

fileinfo = dict()  # count, size
for afile in [conf.FILENAME_NOISE, conf.FILENAME_NORM, conf.FILENAME_OUTL]:
    basename = pathlib.Path(afile).stem
    adir = str(pathlib.Path(conf.DIROUT) / basename)
    files = comm.get_multifiles_indir(adir, "*.csv")
    sizes = comm.get_multifiles_size(files)

    fileinfo[basename] = [len(files), sum(sizes)]

total_count, total_size = 0, 0
for basename, count_size in fileinfo.items():
    print(f"output {basename} files={count_size[0]}, size={human.format_size(count_size[1])}")
    total_count = total_count + count_size[0]
    total_size = total_size + count_size[1]

print(f"total files={total_count}, size={human.format_size(total_size)}")

output ohtnoise files=0, size=0 bytes
output ohtnorm files=3, size=6.7 MB
output ohtoutl files=1, size=2.23 MB
total files=4, size=8.93 MB


In [26]:
_elapsed = time.time() - mainstart
print(f"main elapsed time: {human.format_timespan(_elapsed)}")  # 20 min 27 sec. for conf.INPUT_MAXSIZE=400MB

main elapsed time: 22.78 seconds


## eof